In [1]:
import os, time, math
import traci
import pandas
import numpy as np

## Load data


In [2]:



data_path = "/Users/Res_proj/Documents/Research/120-amsterdam/message-new"
f = []
for (dirpath, dirnames, filenames) in os.walk(data_path):
    f.extend(filenames)
    break
for iter in range(len(f)):
    f[iter] = int(f[iter].split(".")[0])
f.sort()
max_frame = max(f)
for iter in range(len(f)):
    f[iter] = str(f[iter])+'.txt'



Class for vehicles

In [3]:

rotation = 105

class Mobile():
    def __init__(self, type, id, degree,speed, x_br, x_tl, y_br, y_tl):
        x_br = float(x_br)
        x_tl = float(x_tl)
        y_br = float(y_br)
        y_tl = float(y_tl)
        self.type = type
        self.id = id
        self.speed = float(speed)
        self.degree = float(degree)
        self.center = ((x_br+x_tl)/2, (y_br+y_tl)/2)
        self.center = (self.center[0]*np.cos(rotation) + self.center[1]*np.sin(rotation),  -self.center[0]*np.sin(rotation) + self.center[1]*np.cos(rotation))
        self.center = (self.center[0]/55 + 255, self.center[1]/55 + 170)
        if self.type == 'person':
            self.type = 'DEFAULT_PEDTYPE'
        elif self.type == 'bus' or self.type == 'car':
            self.type = "DEFAULT_VEHTYPE"
        else:
            self.type = "DEFAULT_VEHTYPE"
    
    def addVehicle(self):
        traci.vehicle.add(vehID=self.id, routeID='', typeID = self.type , depart="now")
        traci.vehicle.moveToXY(vehID=self.id, edgeID="", lane = "0", x = self.center[0], y = self.center[1], keepRoute=2, angle = self.degree)
        traci.vehicle.setSpeed(vehID=self.id, speed=self.speed)
    
    def updateVehicle(self):
        print(self.center)
        traci.vehicle.moveToXY(vehID=self.id, edgeID="", lane = "0", x = self.center[0], y = self.center[1], keepRoute=2, angle = self.degree)
        traci.vehicle.setSpeed(vehID=self.id, speed=self.speed)

    def removeVehicle(self):
        traci.vehicle.remove(self.id)
        



In [4]:
frame = dict()
for iter in range(len(f)):
    with open(data_path+'/'+f[iter],'r') as input:
        frame_num = int(f[iter].split(".")[0])
        frame[frame_num] = dict()
        filerows = input.readlines()
        for row in filerows:
            type, id, degree, direcion, speed, x_br, x_tl, y_br, y_tl = row.strip().split()
            Veh = Mobile(type, id, degree, speed, x_br, x_tl, y_br, y_tl)
            frame[frame_num][id] = Veh


## start simulation, interval = 6HZ

In [5]:
sumoBinary = os.path.join(os.environ['SUMO_HOME'], 'bin', "sumo-gui")
sumoCmd = [sumoBinary, "-n", "/Users/Res_proj/Documents/Research/120-amsterdam/map/map.net.xml", 
"--step-length", str(1/6),"--start", '--log', './log_file.txt', '--num-clients' , '1', '--delay', '1000']
traci.start(sumoCmd, port = 3400)

 Retrying in 1 seconds


Fontconfig warning: ignoring UTF-8: not a valid region tag


(20, 'SUMO v1_14_1+1789-6cfa3a18e2c')

Main simulation Loop

In [6]:
step = 0
while step <= max_frame:
    time.sleep(0.1)
    print("step", step)
    traci.simulationStep()
    running_car = set(traci.vehicle.getIDList())
    car_in_frame = set(frame[step].keys())
    
    add_car_set = car_in_frame - running_car
    update_car_set = car_in_frame & running_car
    remove_car_set = running_car - car_in_frame
    print(add_car_set)
    print(update_car_set)
    print(remove_car_set)
    for item in add_car_set:
        frame[step][item].addVehicle()
    for item in update_car_set:
        frame[step][item].updateVehicle()
    for item in remove_car_set:
        frame[step-1][item].removeVehicle()
    step +=1

    


    

step 0
{'0', '5', '8', '10', '18', '6', '16', '4', '7', '2', '12', '9', '1', '3', '11', '15'}
set()
set()
step 1
{'19'}
{'7', '0', '5', '2', '8', '4', '10', '18', '1', '11', '15', '3', '9', '16', '6'}
{'12'}
(233.55585733102296, 209.62773239720667)
(222.67091082620135, 219.43946036443418)
(217.13663134266278, 190.91079103446754)
(218.29055666400234, 229.38614143682486)
(220.45712920141756, 184.10116029341344)
(216.7437278686329, 187.92823728640278)
(228.74798724085093, 180.36145549514026)
(240.63336942420008, 187.83650519675948)
(213.57844189848558, 229.19976878009254)
(239.44468451370594, 182.24908543549918)
(239.40969871704152, 183.78593822282122)
(217.02429549977666, 184.1949327453708)
(237.808726486547, 194.49760384863166)
(224.49278267671775, 222.55198470228032)
(221.93527136645935, 190.06955264899577)
step 2
{'12'}
{'0', '5', '2', '8', '4', '10', '18', '1', '11', '15', '3', '9', '16', '19', '6'}
{'7'}
(222.67973387423353, 219.44165090124542)
(217.63759814683343, 191.1944055964998